In [ ]:
import casadi as ca
import numpy as np

def integrator(f: ca.Function, x: ca.SX, u: ca.SX, dt: ca.SX):
    k1 = f(x, u)
    k2 = f(x + 0.5 * dt * k1, u)
    k3 = f(x + 0.5 * dt * k2, u)
    k4 = f(x + dt * k3, u)
    return x + (dt/ 6.0) * (k1 + 2.0 * k2 + 2.0 * k3 + k4)


# symbolic dynamics model for estimation and control
q_BI = ca.SX.sym('q_BI', 4)
omega = ca.SX.sym('omega', 3)
omega_w = ca.SX.sym("omega_w", 3)
x = ca.vertcat(q_BI, omega, omega_w)
u = ca.SX.sym('inputs', 6)
u_rw = u[:3]
u_mag = u[3:]

J_hat = np.eye(3,3)
J_w = np.ones(1)
A_hat = np.eye(3,3)
dt = 0.01

                    
qx, qy, qz, qw = q_BI[0], q_BI[1], q_BI[2], q_BI[3]
wx, wy, wz = omega[0], omega[1], omega[2]
q_dot = 0.5 * ca.vertcat(
    -qx*wx - qy*wy - qz*wz,
    qw*wx + qy*wz - qz*wy,
    qw*wy - qx*wz + qz*wx,
    qw*wz + qx*wy - qy*wx
)

h_int = A_hat @ (J_w * (omega_w + A_hat @ omega)) 
tau_rw = A_hat @ u_rw # electrical dynamics not included
tau_mag = u_mag # TODO: implement

cross_term = ca.cross(omega, J_hat @ omega + h_int)
total_torque = tau_mag - tau_rw - cross_term
omega_dot = ca.solve(J_hat, total_torque)
omega_w_dot = u_rw / J_w - A_hat @ omega_dot

dx = ca.vertcat(q_dot, omega_dot, omega_w_dot)

f = ca.Function("system_dynamics", [x, u], [dx])

x_next = integrator(f, x, u, dt)

F = ca.Function("F", [x, u, dt], [x_next], ['x','u','dt'], ['x_next'])

In [ ]:
import os
from simulation import Simulation


eos_file_path = os.path.join(os.path.dirname(os.getcwd()), "tudsat-trace_eos.json")
    
sim = Simulation.from_json(eos_file_path)
sim.run()

In [7]:
sim.inital_state.shape

(13,)

In [ ]:
ca.SX.zeros(3)

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter
import casadi as ca

# TODO: implement extended kalman filter wrapper
class IMUEKF(ExtendedKalmanFilter):
    def __init__(self, dim_x, dim_z, f: ca.Function, g: ca.Function):
        super().__init__(dim_x, dim_z)

        self.f = f # process model

        dt = ca.SX.sym("dt")
        self.F = integrator(self.f, self.x, u, dt) # function returns next state
        
        self.g = g # measurement model

        self.g_jac = g.jacobian()

    def predict_x(self, u, dt):
        self.x = self.F(self.x, u, dt)

    def update(self, z):

        super().update(z, self.g_jac, self.g)


In [ ]:
import json

eos_path = r"tudsat-trace_eos.json"


with open(eos_path, "r") as f:
    eos_file = json.load(f)

eos_file["ModelObjects"]["Reaction Wheel1"]["Orientation"]

In [ ]:
from actuators import ReactionWheel, Magnetorquer
from kinematics import euler_ocr_to_sbc
from satellite import Spacecraft, Surface, replace_orientation_matrices, string_to_matrix
import json
import datetime
from sensors import *
from simulation import Simulation
from scipy.spatial.transform import Rotation as R

eos_path = r"tudsat-trace_eos.json"


with open(eos_path, "r") as f:
    eos_file = json.load(f)

data = replace_orientation_matrices(eos_file)  

trace = data["ModelObjects"]["TRACE"]

m = trace["StructureMass"]
J_B = string_to_matrix(trace["StructureMomentOfInertia"])

# # TODO: in satellite factory classmethod
surfaces = [Surface.from_eos_panel(v) for k, v in data["ModelObjects"].items() if "panel" in k.lower()]
rws = [ReactionWheel(0.2e-3, 6000, 2e-6, a) for a in [[1, 0, 0], [0, 1, 0], [0, 0, 1]]]
magnetorquers = [Magnetorquer(0.2e-2, a) for a in [[1, 0, 0], [0, 1, 0], [0, 0, 1]]]

dt = 0.01
sun_sensor = SunSensor(dt, 0.0)
magnetometer = Magnetometer(dt, 0.0, [0, 0, 0])
gps = GPS(dt, 0.0)
accelerometer = Accelerometer(dt, 0.0, 0.0)
gyro = Gyroscope(dt, 0.0, 0.0)
rw_speed_sensors = RW_tachometer(dt, 0.0)

Spacecraft(m, J_B, surfaces, rws, magnetorquers, sun_sensor, magnetometer, gps, accelerometer, gyro, rw_speed_sensors)


In [ ]:
import datetime

datetime.datetime(datetime.MINYEAR, 1, 1)

In [ ]:
def simplify_eos(eos_file_path: str, out_path: str):
    with open(eos_file_path, "r") as f:
        eos_file = json.load(f)

    data = replace_orientation_matrices(eos_file)  

    

In [ ]:
from itertools import cycle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.colors import TABLEAU_COLORS


## code for plotting surfaces
surfaces: list[Surface]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')


# solar panel = 10x10cm 
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = cycle(prop_cycle.by_key()['color'])

names = [k for k, v in data["ModelObjects"].items() if "panel" in k.lower()] 
for i, s in enumerate(surfaces):
    c = next(colors)
    s.plot(ax, c, 0.05, 0.05)
    ax.scatter(*s.pos, c=c, label=names[i])
#surfaces[1].plot(ax)
ax.set_ylabel("Y axis")
ax.set_xlabel("X axis")
ax.set_zlabel("Z axis")
print(s.center)
print(s.normal)
print(s.area)
# ax.set_xlim(-2, 2)
# ax.set_ylim(-2, 2)
# ax.set_zlim(-2, 2)
ax.set_aspect("equal")
ax.legend()
fig.tight_layout()

In [ ]:
from io import StringIO
import sys
from IPython.display import Latex
import casadi as ca
import re
import numpy as np


def latex_print(expr, sub=False):
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    expr.print_sparse()
    sys.stdout = old_stdout
    x = mystdout.getvalue().split('\n')

    shape_split = x[0].split(':')[1].split(',')[0].strip().split('-')
    m = int(shape_split[0])
    n = int(shape_split[2])
    e = {}
    b = {
        '*': r' \cdot ',
        'cos': r'\cos',
        'sin': r'\sin'
    }
    for l in ['alpha', 'beta', 'gamma', 'Gamma',
            'delta', 'Delta', 'epsilon', 'zeta', 'eta', 'theta',
            'Theta', 'iota', 'kappa', 'lambda', 'Lambda', 'mu',
            'nu', 'xi', 'pi', 'rho', 'sigma', 'tau', 'upsilon',
            'phi', 'Phi', 'chi', 'psi', 'Psi', 'omega', 'Omega']:
        b[l] = '\\' + l
    s = r'\b(' + '|'.join(sorted(b.keys(), key=len, reverse=True)) + r')\b'
    # fix for escaped keys
    s = s.replace('*', '\\*')
    b_pattern = re.compile(s)
    t = {}
    for i in range(1, len(x)):
        line = x[i].strip()
        if line[0] == "@":
            split = line.split("=")
            term = split[1].replace(',', '')
            term = b_pattern.sub(lambda x: b[x.group()], term)
            e[split[0]] = term
        elif line[0] == "(":
            split = line.split("->")
            indices = split[0].strip().replace('(', '').replace(')', '').split(',')
            i = int(indices[0].strip())
            j = int(indices[1].strip())
            t[(i, j)] = str(split[1])
    s = r"\begin{bmatrix}"
    for i in range(m):
        row = []
        for j in range(n):
            term = "0";
            if (i, j) in t.keys():
                term = t[(i, j)]
            term = b_pattern.sub(lambda x: b[x.group()], term)
            if sub:
                for k in sorted(e.keys(), key=len, reverse=True):
                    term = term.replace(k, e[k])
            row += [r"{:s}".format(term)]
        s += "&".join(row) + r"\\"
    s += r"\end{bmatrix}"
    if not sub:
        s += r"\begin{align}"
        for k in e.keys():
            s += r"{:s}: && {:s} \\".format(k, e[k])
        s += r"\end{align}"
    return s

def latex_display(x, sub=True):
    return Latex(latex_print(x, sub))


# symbolic dynamics model for estimation and control
q_BI = ca.SX.sym('q', 4)
omega = ca.SX.sym(r'\omega', 3)
omega_w = ca.SX.sym(r"\Omega", 3)
x = ca.vertcat(q_BI, omega, omega_w)
u = ca.SX.sym('u', 6)
u_rw = u[:3]
u_mag = u[3:]

B_B = ca.SX.sym('B', 3) # where B is the measured magnetic field from the                   
A_hat = np.eye(3)
J_hat = np.eye(3)
J_w = np.ones(3)
K_t_dash = 1
K_e_rw = 1
K_mag = 1

# actuators
h_int = A_hat @ (J_w * (omega_w + A_hat @ omega)) 
tau_rw = K_t_dash * (A_hat @ (u_rw - K_e_rw * omega_w)) 

# r_norm = ca.norm_2(r_ECI) # instead of dipole model the measurements are used
# r_hat = r_ECI / r_norm
# B_I = (3 * r_hat * (ca.dot(M_dipole, r_hat)) - M_dipole) / r_norm**3
tau_mag = ca.cross(K_mag * u_mag, B_B) 

# attitude kinematics
q_dot = ca.SX.sym(r"\dot{q}", 4)
q_dot[:3] = 0.5 * (omega * q_BI[3] + ca.cross(omega, q_BI[:3]))
q_dot[3] = -0.5 * ca.dot(omega, q_BI[:3])

# attitude dynamics
cross_term = ca.cross(omega, J_hat @ omega + h_int)
total_torque = tau_mag - tau_rw - cross_term
omega_dot = ca.solve(J_hat, total_torque)
# omega_w_dot = u_rw / J_w - A_hat @ omega_dot


dx = ca.vertcat(q_dot, omega_dot)
f = ca.Function("f", [x, u, B_B], [dx], ["x", "u", "B"], ["dx"])

latex_display(dx)

In [ ]:

f_jac = f.jacobian()

# x = 3 * np.ones(10)

# u = 2 * np.ones(6)

# B_B = np.ones(3)

# f_jac([x, u, B_B])

out = np.zeros(7)

A, B, _ = f_jac(x, u, B_B, out)

A